# News Sentiment Analysis

## Import Libraries

In [1]:
# // TODO: TINGS
#     ✓ Find API for data collection
#     ✓ Data Gathering
#     ✓ Data Cleaning
#     - Feature generation
#     - Feature Engineering/selection
#     - Model Train
#     - Model Test
#     - Model Evaluation

In [2]:
# Common Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import glob
import re
from IPython.display import clear_output
import time

# Cleaner output
from tqdm import tqdm

# Deep Learning Libraries
import torch

# Add the path to the API Scraper
## Project Path
project_path = "../"

## Add the path to API Scraper
sys.path.append(os.path.abspath(os.path.join(project_path, "lib")))

# Custom News Libraries
from scraper import get_cached_news_metadata, extract_text_from_url

# Text preprocessing
from preprocessor import clean_text

## Fetch Data

In [3]:
def scrape_data(n_request: int = 10, before_date: str = "2025-12", path: str = ".", last_idx:int = 0) -> pd.DataFrame:
    
    articles = []

    for i in tqdm(range(last_idx, last_idx + n_request), desc="Fetching News Data...", unit="news"):
        metadata = get_cached_news_metadata(page=i, before_date=before_date, path=path)

        data_list = metadata.get("data", [])

        for article in data_list:
            # extract sentiment (first entity if exists)
            entities = article.get("entities", [])
            if entities and "sentiment_score" in entities[0]:
                sentiment = entities[0]["sentiment_score"]
            else:
                sentiment = None

            # store sentiment as its own field inside article
            article["sentiment"] = sentiment

            articles.append(article)

    # final dataframe
    return pd.DataFrame(articles)

### Caching

In [4]:
# Arguments for the caching function
before_date = "2025-11"

data_path = os.path.join(project_path,f"news_cache/{before_date}/csv/")
os.makedirs(data_path, exist_ok=True) # if the directory exist no need to make

cached_file = os.path.join(data_path, f"{before_date}_news_data.csv")

# Set this to TRUE if you want to get new data/overwrite the old data
overwrite_cached_links = True

# get the last index of the file 
json_files = os.path.join(project_path, f"news_cache/{before_date}/json/page-*.json")
list_of_files = glob.glob(json_files)

## Extract page numbers set to True if you want to get new api request.
get_new_requests = False

if get_new_requests:
    page_nums = []
    for f in list_of_files:
        match = re.search(r"page-(\d+)\.json$", os.path.basename(f))
        if match:
            page_nums.append(int(match.group(1)))

    last_idx = max(page_nums)
    n_requests = 101 # marketaux api limit (last index is excluded)

else:
    last_idx = 0
    n_requests = len(list_of_files) #number of pages we cached to be processed

In [5]:
# filling missing text data function

def safe_extract(row):
    try:
        text = extract_text_from_url(row["url"])
        # print(f"{row['title']} {row['description']}")
        return text

    except Exception as e:
        tqdm.write(f"failed to extract from url \n Err: {e} \n appending title and description")
        time.sleep(2)
        print(f"{row['title']} {row['description']}")
        clear_output(wait=True)
        text = f"{row['title']} {row['description']}"
        return text

In [ ]:
# tqdm for cleaner output
tqdm.pandas(desc="Extracting News from URL's", unit="news")

# We will cache the data so that it will load faster
if os.path.exists(cached_file) and not overwrite_cached_links:
    print("Loading cached dataset...")
    news_df = pd.read_csv(cached_file)
    print("Cached dataset loaded")

elif os.path.exists(cached_file) and overwrite_cached_links:
    print("Overwriting old data and caching new data...")
    # Scrape the data
    news_df = scrape_data(n_request=n_requests, before_date= before_date, path= project_path, last_idx=last_idx)
    
    # Extract text from the news
    news_df["text"] = news_df.progress_apply(safe_extract, axis=1)
    news_df.to_csv(cached_file, index=False)
    print("Done Overwriting old data and caching new data...")

else:
    print("Creating and caching dataset...")
    news_df = scrape_data(n_request=n_requests, before_date= before_date, path= project_path, last_idx=last_idx)
    news_df["text"] = news_df.progress_apply(safe_extract, axis=1)
    news_df.to_csv(cached_file, index=False)
    print("Finished Caching")

Extracting News from URL's:  42%|████▏     | 1006/2382 [14:24<19:42,  1.16news/s]  


KeyboardInterrupt: 

### Fetch the Text from URL

## EDA

### View the Data

In [ ]:
news_df.head(10)

,uuid,title,description,keywords,snippet,url,image_url,language,published_at,source,relevance_score,entities,similar,sentiment,text
0,487e6a88-d3c2-4ae1-8dc2-26af6b31d688,2025: The Year Of Alphabet (GOOG),No stock has seen a bigger jump recently than ...,NaN,vzphotos/iStock Editorial via Getty Images\n\n...,https://seekingalpha.com/article/4848680-2025-...,https://static.seekingalpha.com/cdn/s3/uploads...,en,2025-11-30T05:30:00.000000Z,seekingalpha.com,NaN,"[{'symbol': 'GOOGL', 'name': 'Alphabet Inc.', ...",[],0.0000,vzphotos/iStock Editorial via Getty Images\n\n...
1,92b5c2bd-d324-4ae8-b115-2cfd95a8fa98,Why I'm Doubling Down On My Adobe Position (NA...,"Adobe's revenue is highly predictable, driven ...",NaN,To say that Adobe ( ADBE ) stock has not had a...,https://seekingalpha.com/article/4848762-why-i...,https://static.seekingalpha.com/cdn/s3/uploads...,en,2025-11-30T05:25:01.000000Z,seekingalpha.com,NaN,"[{'symbol': 'ADBE', 'name': 'Adobe Inc.', 'exc...",[],0.0000,To say that Adobe ( ADBE ) stock has not had a...
2,9084e5f1-75f5-4f15-aa3d-0676073b4aaf,Global week ahead: The start of a Santa Rally ...,NaN,"STOXX 600, business news",And just like that... December is upon us. It'...,https://www.cnbc.com/2025/11/30/global-week-ah...,https://image.cnbcfm.com/api/v1/image/10823257...,en,2025-11-30T05:10:58.000000Z,cnbc.com,NaN,"[{'symbol': 'M', 'name': ""Macy's, Inc."", 'exch...",[],0.6908,And just like that... December is upon us. It'...
3,487e6a88-d3c2-4ae1-8dc2-26af6b31d688,2025: The Year Of Alphabet (GOOG),No stock has seen a bigger jump recently than ...,NaN,vzphotos/iStock Editorial via Getty Images\n\n...,https://seekingalpha.com/article/4848680-2025-...,https://static.seekingalpha.com/cdn/s3/uploads...,en,2025-11-30T05:30:00.000000Z,seekingalpha.com,NaN,"[{'symbol': 'GOOGL', 'name': 'Alphabet Inc.', ...",[],0.0000,vzphotos/iStock Editorial via Getty Images\n\n...
4,92b5c2bd-d324-4ae8-b115-2cfd95a8fa98,Why I'm Doubling Down On My Adobe Position (NA...,"Adobe's revenue is highly predictable, driven ...",NaN,To say that Adobe ( ADBE ) stock has not had a...,https://seekingalpha.com/article/4848762-why-i...,https://static.seekingalpha.com/cdn/s3/uploads...,en,2025-11-30T05:25:01.000000Z,seekingalpha.com,NaN,"[{'symbol': 'ADBE', 'name': 'Adobe Inc.', 'exc...",[],0.0000,To say that Adobe ( ADBE ) stock has not had a...
5,9084e5f1-75f5-4f15-aa3d-0676073b4aaf,Global week ahead: The start of a Santa Rally ...,NaN,"STOXX 600, business news",And just like that... December is upon us. It'...,https://www.cnbc.com/2025/11/30/global-week-ah...,https://image.cnbcfm.com/api/v1/image/10823257...,en,2025-11-30T05:10:58.000000Z,cnbc.com,NaN,"[{'symbol': 'M', 'name': ""Macy's, Inc."", 'exch...",[],0.6908,And just like that... December is upon us. It'...
6,7d36a275-f3a3-44ea-8cbc-caa0d67749c4,Global Risk Monitor: Week in Review – Nov 28,KEY ISSUES Silver surged 13% for the week and ...,NaN,KEY ISSUES\n\nSilver surged 13% for the week a...,https://global-macro-monitor.com/2025/11/29/gl...,https://global-macro-monitor.com/wp-content/up...,en,2025-11-30T05:07:50.000000Z,global-macro-monitor.com,NaN,"[{'symbol': 'NVDA', 'name': 'NVIDIA Corporatio...",[],-0.3612,KEY ISSUES\n\nSilver surged 13% for the week a...
7,42ba634c-b7ce-491a-91c0-e2b1424af827,"Mcap boost: 7 of top-10 firms gain ₹96,201 cr;...",Market valuations of seven top firms rose by ₹...,NaN,The combined market valuation of seven of the ...,https://www.thehindubusinessline.com/markets/m...,https://bl-i.thgim.com/public/incoming/ji6cih/...,en,2025-11-30T05:04:20.000000Z,thehindubusinessline.com,NaN,"[{'symbol': 'SBKFF', 'name': 'State Bank of In...",[],0.0000,The combined market valuation of seven of the ...
8,47000f09-22ab-4309-9411-c0c738327c25,QQQX: Tax-Efficient Dividends From The Nasdaq-...,Discover why Nuveen NASDAQ 100 Dynamic Overwri...,NaN,"With the rise of covered call ETFs, it can be ...",https://seekingalpha.com/article/4848757-qqqx-...,https://static.seekingalpha.com/cdn/s3/

### Check for Missing Values

In [ ]:
is_na = pd.DataFrame(news_df.isna().sum())
is_na.columns = ["Number_Missing"]
is_na["Missing_Percentage"] = (is_na["Number_Missing"] / len(news_df) * 100)
print(is_na)

                 Number_Missing  Missing_Percentage
uuid                          0            0.000000
title                         0            0.000000
description                 117            4.911839
keywords                   1145           48.068850
snippet                       8            0.335852
url                           0            0.000000
image_url                     7            0.293871
language                      0            0.000000
published_at                  0            0.000000
source                        0            0.000000
relevance_score            2382          100.000000
entities                      0            0.000000
similar                       0            0.000000
sentiment                     0            0.000000
text                        726           30.478589


## Data Cleaning

### Fix Missing Values

In [ ]:
news_df["text"] = news_df["text"].replace("", None)

In [ ]:
is_na = pd.DataFrame(news_df.isna().sum())
is_na.columns = ["Number_Missing"]
is_na["Missing_Percentage"] = (is_na["Number_Missing"] / len(news_df) * 100)
print(is_na)

                 Number_Missing  Missing_Percentage
uuid                          0            0.000000
title                         0            0.000000
description                 117            4.911839
keywords                   1145           48.068850
snippet                       8            0.335852
url                           0            0.000000
image_url                     7            0.293871
language                      0            0.000000
published_at                  0            0.000000
source                        0            0.000000
relevance_score            2382          100.000000
entities                      0            0.000000
similar                       0            0.000000
sentiment                     0            0.000000
text                        726           30.478589


In [ ]:
# Fill the missing text from the url with just the news description. due to the scraper is unable to get the html since the site requires authorization
news_df["text"] = news_df["text"].fillna(
    news_df["title"] + " " + news_df["description"]
)


In [ ]:
is_na = pd.DataFrame(news_df.isna().sum())
is_na.columns = ["Number_Missing"]
is_na["Missing_Percentage"] = (is_na["Number_Missing"] / len(news_df) * 100)
print(is_na)

                 Number_Missing  Missing_Percentage
uuid                          0            0.000000
title                         0            0.000000
description                 117            4.911839
keywords                   1145           48.068850
snippet                       8            0.335852
url                           0            0.000000
image_url                     7            0.293871
language                      0            0.000000
published_at                  0            0.000000
source                        0            0.000000
relevance_score            2382          100.000000
entities                      0            0.000000
similar                       0            0.000000
sentiment                     0            0.000000
text                          6            0.251889


In [ ]:
news_df.duplicated().sum()

np.int64(215)

In [ ]:
news_df = news_df.drop_duplicates()

In [ ]:
news_df.duplicated().sum()

np.int64(0)

### Clean the Text

In [ ]:
# Arguments for the caching function
cleaned_data_path = os.path.join(project_path,f"news_cache/{before_date}/csv/")
os.makedirs(cleaned_data_path, exist_ok=True) # if the directory exist no need to make

clean_cached_file = os.path.join(cleaned_data_path, f"{before_date}_clean_news_data.csv")

# Set this to TRUE if you want to get new data/overwrite the old data
overwrite_clean_data = True

In [ ]:
# tqdm for cleaner output
tqdm.pandas(desc="Cleaning the Text", unit="news")

# We will cache the data so that it will load faster
if os.path.exists(clean_cached_file) and not overwrite_clean_data:
    print("Loading cached dataset...")
    news_df = pd.read_csv(clean_cached_file)
    print("Cached dataset loaded")

elif os.path.exists(clean_cached_file) and overwrite_clean_data:
    print("Overwriting old data and caching new data...")
    # Clean the data
    news_df["clean_text"] = news_df["text"].progress_apply(
                                                        lambda x: clean_text(
                                                            text = x,
                                                            tokenize=False,
                                                            remove_stop_words= True,
                                                            remove_emojis="keep"
                                                            )
                                                        )
    news_df.to_csv(clean_cached_file, index=False)
    print("Done Overwriting old data and caching new data...")

else:
    print("Creating and caching dataset...")
    # Clean the data
    news_df["clean_text"] = news_df["text"].progress_apply(
                                                        lambda x: clean_text(
                                                            text = x,
                                                            tokenize=False,
                                                            remove_stop_words= True,
                                                            remove_emojis="keep"
                                                            )
                                                        )
    news_df.to_csv(clean_cached_file, index=False)
    print("Finished Caching")

Overwriting old data and caching new data...


Cleaning the Text:  25%|██▍       | 535/2167 [00:00<00:01, 1429.35news/s]


AttributeError: 'float' object has no attribute 'lower'